### G_EVAL 테스트 데이터셋 생성
- 10개 회사별로 각각 10개씩 `파싱된 문서`를 랜덤으로 불러온 후 100개 문서에 대해서 QnA 생성

    - `파싱된 문서`의 버전은 PDF 원문만 나와있는 것으로 진행: final_documents_ver2.pkl

        - notebooks/RAG/data_preprocessing/data_preprocessing_ver2.ipynb의 결과 파일

- `난이도별`로 (쉬움: 40개, 보통: 30개, 어려움: 30개) QnA 생성

    - 쉬움과 보통 난이도는 `하나의 문서`에서 대해서 QnA 생성

    - 어려움 난이도는 `두 개의 문서`에서 대해서 QnA 생성


In [2]:
# Cell 1: 기본 준비 - 회사별로 10개씩 랜덤하게 파싱된 문서를 추출

# 데이터 로드 및 회사별 문서 선택
import pickle
import random
from collections import defaultdict
import pandas as pd
from typing import List, Dict
from langchain.schema import Document

# 재현성을 위한 시드 설정
random.seed(42)

# 데이터 로드
with open('./data/final_documents_ver2.pkl', 'rb') as f:
    documents = pickle.load(f)

# 회사별로 문서 분류
company_docs = defaultdict(list)
for doc in documents:
    company = doc.metadata.get('company_name')
    if company:
        company_docs[company].append(doc)

# 각 회사별로 10개씩 랜덤 선택
selected_docs = []
for company in ['네이버', '크래프톤', 'CJ제일제당', 'LG화학', 'SK케미칼', 
                'SK하이닉스', '롯데렌탈', '엘앤에프', '카카오뱅크', '한화솔루션']:
    if company in company_docs:
        docs = random.sample(company_docs[company], 10)
        selected_docs.extend(docs)

print(f"선택된 총 문서 수: {len(selected_docs)}")
for company in set(doc.metadata['company_name'] for doc in selected_docs):
    count = sum(1 for doc in selected_docs if doc.metadata['company_name'] == company)
    print(f"{company}: {count}개")

선택된 총 문서 수: 100
카카오뱅크: 10개
LG화학: 10개
네이버: 10개
SK하이닉스: 10개
SK케미칼: 10개
CJ제일제당: 10개
크래프톤: 10개
엘앤에프: 10개
한화솔루션: 10개
롯데렌탈: 10개


In [6]:
# Cell 2: 쉬움/보통 난이도 QA 쌍 생성 (40개 쉬움 + 30개 보통)
from openai import OpenAI
import json

def generate_simple_qa(document: Document, difficulty: str, client: OpenAI) -> Dict:
    """단일 문서에 대한 QA 쌍 생성"""
    
    # 메타데이터 추출
    company = document.metadata.get('company_name')
    broker = document.metadata.get('broker')
    date = document.metadata.get('date')  # YYYYMMDD 형식
    
    if difficulty == "쉬움":
        prompt = f"""
금융 리포트 내용을 바탕으로 쉬운 난이도의 질문-답변 쌍을 하나만 생성해주세요.
반드시 다음 두 가지 유형 중 하나로만 생성하세요:

1. 투자의견/목표가 관련:
   - 예시: "{date} 발행된 {broker}의 {company}에 대한 투자의견과 목표주가는 얼마인가요?"
   
2. 실적 관련 단순 수치:
   - 예시: "{date} 발행된 보고서 기준, 2024년 {company}의 예상 영업이익은 얼마인가요?"

리포트 내용:
---
{document.page_content}
---

주의사항:
1. 반드시 리포트에서 명확하게 확인할 수 있는 내용만 다룰 것
2. 답변은 사실관계가 명확한 내용만 포함할 것
3. 리포트의 발행일자({date})가 나온다면 포함시킬 것
"""
    else:  # 보통 난이도
        prompt = f"""
금융 리포트 내용을 바탕으로 보통 난이도의 질문-답변 쌍을 하나만 생성해주세요.
반드시 다음 두 가지 유형을 참고해서 만드세요.

1. 실적 추세 관련:
   - 예시: "{date} 발행된 보고서에 따르면, {company}의 2023년 대비 2024년 영업이익 증감률은 얼마로 전망되나요?"
   
2. 산업 동향 관련:
   - 예시: "{date} 발행된 보고서에서 분석한 {company}의 주요 산업 환경 변화는 무엇인가요?"

리포트 내용:
---
{document.page_content}
---

주의사항:
1. 반드시 리포트에서 명확하게 확인할 수 있는 내용만 다룰 것
2. 답변은 사실관계가 명확한 내용만 포함할 것
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 증권사 애널리스트입니다. 리포트의 내용에서 확실한 정보만을 바탕으로 QA를 생성해주세요."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "qa_pair_generation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "생성된 질문"
                        },
                        "answer": {
                            "type": "string",
                            "description": "300자 이내의 답변"
                        }
                    },
                    "required": ["question", "answer"],
                    "additionalProperties": False
                }
            }
        }
    )
    
    result = json.loads(response.choices[0].message.content)
    return {
        "question": result['question'],
        "groundtruth": result['answer']
    }

# 문서 무작위 섞기
random.shuffle(selected_docs)

# QA 쌍 생성
client = OpenAI()
qa_pairs = []

# 쉬움 난이도 40개
print("쉬움 난이도 QA 쌍 생성 중...")
for i, doc in enumerate(selected_docs[:40]):
    qa = generate_simple_qa(doc, "쉬움", client)
    qa_pairs.append(qa)
    if (i + 1) % 10 == 0:
        print(f"{i + 1}/40 완료")

# 보통 난이도 30개
print("\n보통 난이도 QA 쌍 생성 중...")
for i, doc in enumerate(selected_docs[40:70]):
    qa = generate_simple_qa(doc, "보통", client)
    qa_pairs.append(qa)
    if (i + 1) % 10 == 0:
        print(f"{i + 1}/30 완료")

# 결과 확인
print(f"\n생성된 총 QA 쌍: {len(qa_pairs)}개")
print("\n샘플 QA 쌍:")
for i in range(min(10, len(qa_pairs))):
    print(f"\n질문 {i+1}: {qa_pairs[i]['question']}")
    print(f"답변 {i+1}: {qa_pairs[i]['groundtruth']}")
    print(f"답변 길이: {len(qa_pairs[i]['groundtruth'])}자")

쉬움 난이도 QA 쌍 생성 중...


10/40 완료
20/40 완료
30/40 완료
40/40 완료

보통 난이도 QA 쌍 생성 중...
10/30 완료
20/30 완료
30/30 완료

생성된 총 QA 쌍: 70개

샘플 QA 쌍:

질문 1: 20241021에 IBK투자증권에서 발행된 보고서 기준, CJ제일제당의 투자의견과 목표주가는 얼마인가요?
답변 1: 리포트 내용에 CJ제일제당의 투자의견과 목표주가는 언급되어 있지 않습니다.
답변 길이: 41자

질문 2: 하나증권에서 발행된 2024년 LG화학의 예상 영업이익은 얼마인가요?
답변 2: 2024년 LG화학의 예상 영업이익은 1,056.5입니다.
답변 길이: 32자

질문 3: 20240813에 발행된 DS투자증권의 리포트에 따르면, 2026년 기준 엘앤에프의 예상 고객사별 비중은 어떻게 되나요?
답변 3: 2026년 기준 예상 고객사별 비중은 LG에너지솔루션 41%, SK온 17%, 테슬라 33%, 기타 9%로 다각화가 확인될 전망입니다.
답변 길이: 75자

질문 4: 20240813에 발행된 DS투자증권의 엘앤에프에 대한 영업이익은 얼마인가요?
답변 4: 2024년 엘앤에프의 예상 영업이익은 -368억원입니다.
답변 길이: 31자

질문 5: 20241107에 발행된 하나증권의 카카오뱅크에 대한 투자의견은 무엇인가요?
답변 5: BUY(매수)입니다.
답변 길이: 11자

질문 6: 2024년 롯데렌탈의 예상 영업이익은 얼마인가요?
답변 6: 2024년 롯데렌탈의 예상 영업이익은 296십억원입니다. (SK증권 리포트 기준, 2024년 12월 9일 발행)
답변 길이: 62자

질문 7: 2024년 크래프톤의 예상 유동자산은 얼마인가요?
답변 7: 2024년 크래프톤의 예상 유동자산은 4,681십억원입니다. (미래에셋증권 리포트 발행일: 20240509)
답변 길이: 60자

질문 8: 20240709에 발행된 NH투자증권의 SK케미칼에 대한 투자의견과 목표주가는 얼마인가요?
답변 8: NH투자증권에서 발행한 20240709 리포트에서는 

In [7]:
# Cell 3: 어려움 난이도 QA 쌍 생성 (30개)
from itertools import combinations

def generate_complex_qa(doc1: Document, doc2: Document, client: OpenAI) -> Dict:
    """두 개의 문서를 조합하여 어려운 난이도의 QA 쌍 생성"""
    
    # 메타데이터 추출
    company1 = doc1.metadata.get('company_name')
    company2 = doc2.metadata.get('company_name')

    prompt = f"""
두 금융 리포트의 내용을 종합적으로 분석하여 어려운 난이도의 질문-답변 쌍을 생성해주세요.
반드시 다음 네 가지 유형 참고해서 생성하세요:

1. 실적 비교 분석:
   - 예시: "{company1}와 {company2}의 2024년 예상 실적 중 더 높은 성장이 기대되는 기업은 어디이며, 그 이유는 무엇인가요?"

2. 산업 영향도 비교:
   - 예시: "현재 직면한 시장 환경이 {company1}와 {company2}의 사업에 미치는 영향은 각각 어떻게 다른가요?"

3. 투자포인트/전략 비교:
   - 예시: "{company1}와 {company2}의 중장기 성장 전략 중 시장 영향력이 더 클 것으로 예상되는 것은 무엇이며 근거는 무엇인가요?"

4. 리스크 요인 비교:
   - 예시: "{company1}와 {company2}가 직면한 주요 리스크 요인들을 비교했을 때, 어느 기업이 더 효과적인 대응 전략을 가지고 있나요?"

회사1 리포트:
---
{doc1.page_content}
---

회사2 리포트:
---
{doc2.page_content}
---

주의사항:
1. 두 문서의 정보를 모두 활용하여 답변 가능한 질문을 생성할 것
2. 단순 비교가 아닌 분석적 사고가 필요한 질문을 생성할 것
3. 답변은 리포트 내용에서 명확한 근거를 가질 것
4. 답변은 300자 이내로 작성할 것
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 증권사 애널리스트입니다. 두 기업을 비교 분석하여 인사이트를 도출하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "qa_pair_generation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "생성된 질문"
                        },
                        "answer": {
                            "type": "string",
                            "description": "300자 이내의 답변"
                        }
                    },
                    "required": ["question", "answer"],
                    "additionalProperties": False
                }
            }
        }
    )
    
    result = json.loads(response.choices[0].message.content)
    return {
        "question": result['question'],
        "groundtruth": result['answer']
    }

# 30개의 문서 페어 생성 (무작위)
doc_pairs = list(combinations(selected_docs[70:], 2))  # 70번째 이후의 문서들 사용
selected_pairs = random.sample(doc_pairs, 30)

# QA 쌍 생성
client = OpenAI()
complex_qa_pairs = []

print("어려움 난이도 QA 쌍 생성 중...")
for i, (doc1, doc2) in enumerate(selected_pairs):
    qa = generate_complex_qa(doc1, doc2, client)
    complex_qa_pairs.append(qa)
    if (i + 1) % 10 == 0:
        print(f"{i + 1}/30 완료")

# 결과 확인
print(f"\n생성된 복잡한 QA 쌍: {len(complex_qa_pairs)}개")
print("\n샘플 QA 쌍:")
for i in range(min(3, len(complex_qa_pairs))):
    print(f"\n질문 {i+1}: {complex_qa_pairs[i]['question']}")
    print(f"답변 {i+1}: {complex_qa_pairs[i]['groundtruth']}")
    print(f"답변 길이: {len(complex_qa_pairs[i]['groundtruth'])}자")

# 모든 QA 쌍 합치기 (기존 qa_pairs + complex_qa_pairs)
all_qa_pairs = qa_pairs + complex_qa_pairs

# 최종 데이터셋 생성
final_dataset = [
    {"id": i + 1, "question": qa["question"], "groundtruth": qa["groundtruth"]}
    for i, qa in enumerate(all_qa_pairs)
]

# CSV 저장
df = pd.DataFrame(final_dataset)
df.to_csv('financial_g_eval_test_dataset.csv', index=False, encoding='utf-8')
print("\n최종 데이터셋이 'financial_g_eval_test_dataset.csv'로 저장되었습니다.")

어려움 난이도 QA 쌍 생성 중...
10/30 완료
20/30 완료
30/30 완료

생성된 복잡한 QA 쌍: 30개

샘플 QA 쌍:

질문 1: SK케미칼의 화학적 재활용 copolyester 상업화가 엘앤에프의 성장 전략에 미치는 영향은 무엇이며, 두 기업의 시장 경쟁력에 어떤 차이를 가져올 것인가요?
답변 1: SK케미칼이 화학적 재활용 copolyester 상업화로 2025년까지 50% 판매 비중 목표를 세운 것은 지속 가능한 제품에 대한 수요 증가와 맞물려 시장에서의 경쟁력을 높일 것으로 예상된다. 반면 엘앤에프는 리튬 이온 배터리 소재 시장에서의 성장을 도모하고 있으며, 두 기업 모두 친환경 제품에 집중하고 있지만, SK케미칼의 공격적인 재활용 전략은 환경 규제가 강화되는 상황에서 차별화된 경쟁력을 제공할 가능성이 크다.
답변 길이: 236자

질문 2: 카카오뱅크와 CJ제일제당의 2024년 실적 전망을 비교할 때, 성장성이 더 높은 기업은 어디이며 그 이유는 무엇인가요?
답변 2: 2024년 카카오뱅크의 순이자손익이 1,254억원으로 예상되며, 이는 전년 대비 12.4% 증가한 수치입니다. 반면 CJ제일제당의 연간 실적은 명시되지 않았지만, 시장 예상 주가 상승률은 10%에서 20% 사이로 보고됩니다. 카카오뱅크는 지속적인 이자수익 증가와 대손충당금 감소로 실적 성장이 기대되며, 안정적인 금융 상품 포트폴리오를 통해 빠른 성장을 이끌 것으로 예상됩니다. 따라서 카카오뱅크가 더 높은 성장성을 보일 것으로 판단됩니다.
답변 길이: 246자

질문 3: SK케미칼과 SK하이닉스의 2024년 예상 실적을 비교할 때, 더 높은 영업활동 현금흐름을 기록할 것으로 보이는 기업은 어디이며, 그 근거는 무엇인가요?
답변 3: 2024년 SK하이닉스는 영업활동 현금흐름이 43,441.8억원으로 예상되며, 이는 2023년 4,278.2억원에서 대폭 증가하는 수치입니다. 반면, SK케미칼의 관련 정보는 제공되지 않아 직접 비교가 어렵습니다. 따라서 SK하이닉스가 더